In [39]:
import pandas as pd
import numpy as np


In [5]:
pred_noise = pd.read_pickle("pred_noise_18.pkl")
pred_noise.head()

,Address,noise_prob
0,286 EAST 3 STREET,0.186736
1,288 EAST 3 STREET,0.202002
2,290 EAST 3 STREET,0.202834
3,292 EAST 3 STREET,0.095283
4,298 EAST 3 STREET,0.651990


In [6]:
pred_ele = pd.read_pickle("pred_ele_18.pkl")
pred_ele.head()

,Address,ele_prob
0,286 EAST 3 STREET,0.000184
1,288 EAST 3 STREET,0.000567
2,290 EAST 3 STREET,0.001005
3,306 EAST 3 STREET,0.193876
4,316 EAST 3 STREET,0.210276


In [7]:
pred_heat = pd.read_pickle("pred_heat_18.pkl")
pred_heat.head()

,Address,heat_prob
0,286 EAST 3 STREET,0.370389
1,288 EAST 3 STREET,0.365438
2,290 EAST 3 STREET,0.344592
3,292 EAST 3 STREET,0.290927
4,298 EAST 3 STREET,0.102429


In [12]:
print(pred_heat.shape)
print(pred_noise.shape)

pred_heat_noise = pd.merge(pred_heat, pred_noise, on='Address', how='outer')
print(pred_heat_noise.shape)

pred_all = pd.merge(pred_heat_noise, pred_ele, on='Address', how='outer')
print(pred_all.shape)
pred_all.head()

(21905, 2)
(21905, 2)
(21905, 3)
(21905, 4)


,Address,heat_prob,noise_prob,ele_prob
0,286 EAST 3 STREET,0.370389,0.186736,0.000184
1,288 EAST 3 STREET,0.365438,0.202002,0.000567
2,290 EAST 3 STREET,0.344592,0.202834,0.001005
3,292 EAST 3 STREET,0.290927,0.095283,NaN
4,298 EAST 3 STREET,0.102429,0.651990,NaN


In [48]:
pred_all.describe()

,heat_prob,noise_prob,ele_prob
count,21905.000000,21905.000000,6.027000e+03
mean,0.314962,0.087986,1.692395e-01
std,0.255586,0.148790,2.806381e-01
min,0.007967,0.001573,2.615841e-11
25%,0.117863,0.020117,1.425998e-04
50%,0.235435,0.039201,1.730110e-02
75%,0.429998,0.080575,1.989721e-01
max,1.000000,1.000000,1.000000e+00


In [53]:
def predict_vio(apt_site):
    apt = (apt_site.split('/')[4])
    building_low = apt.replace('-', ' ')
    manhattan1 = "new_york"
    manhattan2 = "manhattan"
    manhattan_building1 = building_low.find(manhattan1)
    manhattan_building2 = building_low.find(manhattan2)
    if manhattan_building1==-1 & manhattan_building2==-1:
        print("I don't know anything about this building")
    else:
        address_low1=building_low.replace(manhattan1, '')
        address_low2=address_low1.replace(manhattan2, '')
        address_upper1 = address_low2.upper()
        address_upper2 = address_upper1.rstrip()
        print(address_upper2)
        building_row = pred_all.loc[pred_all['Address'] == address_upper2]
        if len(building_row)==0:
            print("I don't know anything about this building")
        else:
            vio_heat = round(building_row.iloc[0][1],2)
            vio_noise = round(building_row.iloc[0][2],2)
            vio_ele = round(building_row.iloc[0][3],2)
            if np.isnan(vio_ele)==True:
                print("the probability of a heat problem is", vio_heat)
                print("the probability of a noise problem is", vio_noise)
                print("this building doesnt have an elevator")
            else:
                print("the probability of a heat problem is", vio_heat)
                print("the probability of a noise problem is", vio_noise)
                print("the probability of an elevator problem is", vio_ele)
            
apt_site = str( input("Enter the streeteasy url: ") )


predict_vio(apt_site)

Enter the streeteasy url: https://streeteasy.com/building/66-pinehurst-avenue-new_york/d1
66 PINEHURST AVENUE
the probability of a heat problem is 0.33
the probability of a noise problem is 0.17
this building doesnt have an elevator
